In [1]:
import os
import gc
import numpy as np
import json
import torch
from torch.utils.data import DataLoader
from torchinfo import summary

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

from src.utils import *
from src.runner import run_config

from src.models import UNet, UNet2, UNet4, UNet8, UNetShallow, UNet2Shallow, UNet4Shallow, UNet8Shallow
from src.model_parts import SingleConv, DoubleConv, TripleConv

c:\Users\wittd\anaconda3\Lib\site-packages\pytools\persistent_dict.py:63: RecommendedHashNotFoundWarning: Unable to import recommended hash 'siphash24.siphash13', falling back to 'hashlib.sha256'. Run 'python3 -m pip install siphash24' to install the recommended hash.
  warn("Unable to import recommended hash 'siphash24.siphash13', "


Cupy implementation is not available. Make sure you have the right version of Cupy and CUDA installed.
Optional dependecy Dask_image is not installed. Implementations using it will be ignored.


c:\Users\wittd\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


tiling_rate = 4
overlap_rate = 0.25

tiling_rate = 8
overlap_rate = 0.3 vagy 0.2

tiling_rate = 10
overlap_rate = 0.25

tiling_rate = 16
overlap_rate = 0.2

In [2]:
config = {
'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh_tile8_len8_doubleconv',
    'project_name': 'Tile8',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNetShallow,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 1,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 10,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.25,

    'dilation': 0,
}

test_tiling(config)
run_config(config)

Tiling test
Bio tiles: torch.Size([169, 8, 8, 8]), Mask tiles: torch.Size([169, 8, 8])

Using device cuda
Datasets created!
Starting run 1...
UNetShallow


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: wdavid (wdave). Use `wandb login --relogin` to force relogin


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.4735793173313141, Detection rate: 0.4269957983193277


Epoch 1/20: 100%|██████████| 104/104 [00:31<00:00,  3.26img/s, loss (batch)=1.06]


...

Epoch 20/20: 100%|██████████| 104/104 [00:32<00:00,  3.24img/s, loss (batch)=0.663]


Checkpoint 20 saved!


detection_rate,▁▇▆▆██▇▇▇▇▇▇▇▇▇█▇▇█▇
epoch,▁▁▁▁▁▂▂▂▂▂▂▂▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
learning rate,██████████████████▁▁
step,▁▁▁▁▂▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇██
train loss,█▄▄▃▂▂▂▂▂▁▂▂▂▂▂▂▁▁▂▁▁▂▁▂▁▁▁▁▂▂▁▁▁▁▂▁▁▁▁▁
validation Dice,▁▃▆▆▅▅▇▇▇▇▇▇██▇▇▇▇▇█
detection_rate,1.00059
epoch,20
learning rate,0.001
step,160
train loss,0.66347




Best epoch: 20


Test metrics: {
    "dice_score": 0.5822243690490723,
    "detection_rate": 0.8958738808875049,
    "avg_pred_cell_size": 1443.352161633717,
    "std_pred_cell_size": 809.1715839147315,
    "avg_mask_cell_size": 1166.066562864928,
    "std_mask_cell_size": 746.0676011982475
}
Run 5 finished!
All runs finished!


In [3]:
config2 = config.copy()
config2['run_name'] = 'UNetSh_tile8_len8_singleconv'
config2['down_conv'] = config2['up_conv'] = SingleConv
test_tiling(config2)
run_config(config2)

Tiling test
Bio tiles: torch.Size([169, 8, 8, 8]), Mask tiles: torch.Size([169, 8, 8])

Using device cuda
Datasets created!
Starting run 1...
UNetShallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.4647841155529022, Detection rate: 0.03230042016806723


Epoch 1/20: 100%|██████████| 104/104 [00:36<00:00,  2.85img/s, loss (batch)=0.869]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:26<00:00,  3.88img/s, loss (batch)=0.703]


Checkpoint 20 saved!


detection_rate,▁▅█▇▇███▇▆████▇▇▇███
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇████
learning rate,█████████████▁▁▁▁▁▁▁
step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇█████
train loss,█▇▅▂▂▂▄▂▂▁▂▁▂▂▁▃▂▂▂▂▂▂▁▁▂▂▂▂▁▂▁▂▁▁▁▁▁▁▁▁
validation Dice,▁▆▇▇▇▇▇█▇▇▇▇▇███████
detection_rate,0.9479
epoch,20
learning rate,0.001
step,160
train loss,0.70323




Best epoch: 20


Test metrics: {
    "dice_score": 0.5773302912712097,
    "detection_rate": 0.8333982094200078,
    "avg_pred_cell_size": 1462.809434843531,
    "std_pred_cell_size": 860.642689492475,
    "avg_mask_cell_size": 1166.066562864928,
    "std_mask_cell_size": 746.0676011982475
}
Run 5 finished!
All runs finished!


In [4]:
config3 = config.copy()
config3['run_name'] = 'UNetSh4_tile8_len8_doubleconv'
config3['down_conv'] = config3['up_conv'] = DoubleConv
config3['model'] = UNet4Shallow
config3['mask_scale'] = 4
test_tiling(config3)
run_config(config3)

Tiling test
Bio tiles: torch.Size([169, 8, 8, 8]), Mask tiles: torch.Size([169, 32, 32])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.43878015875816345, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:29<00:00,  3.57img/s, loss (batch)=1.11]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:26<00:00,  3.91img/s, loss (batch)=0.735]


Checkpoint 20 saved!


detection_rate,▁▂▁▅▇▅█▇▇▆█▆▇▇▇▇▆█▇▆
epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇▇██
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇█████
train loss,█▄▄▃▅▂▂▂▂▂▂▂▁▂▂▂▁▂▁▂▁▂▁▂▁▂▁▁▁▂▂▂▂▁▁▂▁▂▂▁
validation Dice,▁▂▁▄▅▅▄▆▇▇▇▆▇▇█▇▇▇██
detection_rate,0.69362
epoch,20
learning rate,0.01
step,160
train loss,0.73461




Best epoch: 19


Test metrics: {
    "dice_score": 0.536535382270813,
    "detection_rate": 0.6522646964343077,
    "avg_pred_cell_size": 1111.7566486087171,
    "std_pred_cell_size": 859.6554476177755,
    "avg_mask_cell_size": 563.5578119980726,
    "std_mask_cell_size": 513.7853947401379
}
Run 5 finished!
All runs finished!


In [5]:
config4 = config3.copy()
config4['run_name'] = 'UNetSh4_tile8_len8_singleconv'
config4['down_conv'] = config4['up_conv'] = SingleConv
test_tiling(config4)
run_config(config4)

Tiling test
Bio tiles: torch.Size([169, 8, 8, 8]), Mask tiles: torch.Size([169, 32, 32])

Using device cuda
Datasets created!
Starting run 1...
UNet4Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.43878015875816345, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:29<00:00,  3.57img/s, loss (batch)=1.06]


Checkpoint 1 saved!


...

Epoch 20/20: 100%|██████████| 104/104 [00:26<00:00,  3.95img/s, loss (batch)=0.748]


Checkpoint 20 saved!


detection_rate,▁▅▇▇█▇▇▆▅▆▆▇▆▇█▇▇██▇
epoch,▁▁▁▁▂▂▂▂▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▇▇▇▇▇▇▇███
learning rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
step,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,█▇▇▅▅▂▃▃▃▃▃▃▃▃▂▂▃▂▂▃▂▁▃▂▂▃▂▂▂▂▁▁▂▁▂▂▂▂▂▂
validation Dice,▁▃▄▃▅▇▇▇▇▇▇▇▇▇▇▇██▇█
detection_rate,0.76741
epoch,20
learning rate,0.01
step,160
train loss,0.74836




Best epoch: 20


Test metrics: {
    "dice_score": 0.5219720602035522,
    "detection_rate": 0.6863154513331192,
    "avg_pred_cell_size": 1213.771428153522,
    "std_pred_cell_size": 993.3548692489549,
    "avg_mask_cell_size": 563.5578119980726,
    "std_mask_cell_size": 513.7853947401379
}
Run 5 finished!
All runs finished!


In [6]:
config5 = config.copy()
config5['run_name'] = 'UNetSh8_tile8_len8_singleconv'
config5['down_conv'] = config5['up_conv'] = SingleConv
config5['model'] = UNet8Shallow
config5['mask_scale'] = 8
test_tiling(config5)
run_config(config5)

Tiling test
Bio tiles: torch.Size([169, 8, 8, 8]), Mask tiles: torch.Size([169, 64, 64])

Using device cuda
Datasets created!
Starting run 1...
UNet8Shallow


Starting training:
        Epochs:          20
        Batch size:      13
        Learning rate:   0.01
        Training size:   104
        Validation size: 26
        Device:          cuda
        Mixed Precision: True
    


Validation Dice score: 0.4303595721721649, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:30<00:00,  3.38img/s, loss (batch)=1.08]


Checkpoint 1 saved!


...

Validation Dice score: 0.3618570566177368, Detection rate: 0.0


Epoch 1/20: 100%|██████████| 104/104 [00:29<00:00,  3.55img/s, loss (batch)=1.14]


Checkpoint 1 saved!


Epoch 2/20: 100%|██████████| 104/104 [00:27<00:00,  3.83img/s, loss (batch)=1.03]


error: Caught error in DataLoader worker process 1.
Original Traceback (most recent call last):
  File "c:\Users\wittd\anaconda3\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wittd\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\wittd\anaconda3\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "c:\onlab_git\Cardio-cell-segmentation\src\dataset.py", line 242, in __getitem__
    bio, mask = get_data_from_file(self.files[index], self.bio_length)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\onlab_git\Cardio-cell-segmentation\src\dataset.p

In [ ]:
config6 = config5.copy()
config6['run_name'] = 'UNetSh8_tile8_len8_doubleconv'
config6['down_conv'] = config6['up_conv'] = DoubleConv
test_tiling(config6)
run_config(config6)

In [ ]:
config = {
'data_path': 'C:/7_felev/data',
    'save_path': 'C:/7_felev/szakdoga',
    # Each run name should be unique, even if the same model is used 
    # or it will overwrite the previous run
    'run_name': 'UNetSh_tile5_len8_doubleconv',
    'project_name': 'Tile5',
    'wandb_logging': True,

    # Unet, UNet2, UNet4, UNet8, UNetShallow, Unet2Shallow, UNet4Shallow, UNet8Shallow
    'model': UNetShallow,
    'down_conv': DoubleConv,
    'up_conv': DoubleConv,
    'bilinear': False,

    'biosensor_length': 8,
    'input_scale': 1,
    # for the imput scaling
    'SRRF_mode': 'eSRRF', # 'eSRRF', 'SRRF', None
    'mask_scale': 1,
    
    'batch_size': 13,
    'epochs': 20,
    'learning_rate': 0.01,
    # K-fold cross validation
    'k': 5,

    'mask_type': bool,
    'augment': True,
    'normalize': True,
    'shuffle': True,
    
    'tiling': True,
    'tiling_ratio': 16,
    # tile size = (80 * input_scale) / tiling_ratio
    'overlap_rate': 0.2,

    'dilation': 0,
}

test_tiling(config)
run_config(config)

In [ ]:
config2 = config.copy()
config2['run_name'] = 'UNetSh_tile5_len8_singleconv'
config2['down_conv'] = config2['up_conv'] = SingleConv
test_tiling(config2)
run_config(config2)

In [ ]:
config3 = config.copy()
config3['run_name'] = 'UNetSh4_tile5_len8_doubleconv'
config3['down_conv'] = config3['up_conv'] = DoubleConv
config3['model'] = UNet4Shallow
config3['mask_scale'] = 4
test_tiling(config3)
run_config(config3)

In [ ]:
config4 = config3.copy()
config4['run_name'] = 'UNetSh4_tile5_len8_singleconv'
config4['down_conv'] = config4['up_conv'] = SingleConv
test_tiling(config4)
run_config(config4)

In [ ]:
config5 = config.copy()
config5['run_name'] = 'UNetSh8_tile5_len8_singleconv'
config5['down_conv'] = config5['up_conv'] = SingleConv
config5['model'] = UNet8Shallow
config5['mask_scale'] = 8
test_tiling(config5)
run_config(config5)

In [ ]:
config6 = config5.copy()
config6['run_name'] = 'UNetSh8_tile5_len8_doubleconv'
config6['down_conv'] = config6['up_conv'] = DoubleConv
test_tiling(config6)
run_config(config6)